In [21]:
!pip install efficientnet_pytorch

In [25]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch
import torch.nn as nn
from PIL import Image
from efficientnet_pytorch import EfficientNet

In [26]:
class DiabeticRetinopathyCSVBasedDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, limit_per_class=None):
        """
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images, expected to have subdirectories named after labels.
        transform (callable, optional): Optional transform to be applied on a sample.
        limit_per_class (int, optional): If provided, limits the number of samples per class.
        """
        self.transform = transform
        self.data = []
        df = pd.read_csv(csv_file)

        if limit_per_class:
            df = df.groupby('diagnosis').apply(lambda x: x.sample(n=limit_per_class, replace=False)).reset_index(drop=True)

        categories = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']  # Make sure these match your folder names exactly

        for _, row in df.iterrows():
            filename = row['id_code'] + '.png'
            label = row['diagnosis']
            category_folder = categories[label]
            img_path = os.path.join(root_dir, category_folder, filename)
            if os.path.exists(img_path):  # Ensure the file exists
                self.data.append((img_path, label))
            else:
                print(f"Warning: {img_path} does not exist.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [27]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize dataset
dataset = DiabeticRetinopathyCSVBasedDataset(csv_file='train.csv', root_dir='gaussian_filtered_images/gaussian_filtered_images', transform=transform, limit_per_class=200)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from efficientnet_pytorch import EfficientNet

# Initialize EfficientNet model
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {100 * correct / total}%')


Epoch 1, Train Loss: 1.5055032968521118, Validation Loss: 1.3075861930847168, Validation Accuracy: 33.333333333333336%
Epoch 2, Train Loss: 0.935589924454689, Validation Loss: 1.3084545135498047, Validation Accuracy: 36.666666666666664%
Epoch 3, Train Loss: 0.5309537649154663, Validation Loss: 1.495376467704773, Validation Accuracy: 30.0%
Epoch 4, Train Loss: 0.280150406062603, Validation Loss: 1.8387808799743652, Validation Accuracy: 30.0%
Epoch 5, Train Loss: 0.06465767417103052, Validation Loss: 1.986570119857788, Validation Accuracy: 30.0%
Epoch 6, Train Loss: 0.08745521632954478, Validation Loss: 2.1749091148376465, Validation Accuracy: 30.0%
Epoch 7, Train Loss: 0.03672278416343033, Validation Loss: 2.4725263118743896, Validation Accuracy: 33.333333333333336%
Epoch 8, Train Loss: 0.027514177840203047, Validation Loss: 2.739529848098755, Validation Accuracy: 40.0%
Epoch 9, Train Loss: 0.07451496552675962, Validation Loss: 2.839116096496582, Validation Accuracy: 36.666666666666664%